In [74]:
import pandas as pd

# 1. test 데이터 불러오기
test_df = pd.read_parquet("../../data/통합_test_데이터.parquet")

In [76]:
from modules.data_loader import map_categorical_columns
from sklearn.preprocessing import LabelEncoder
from modules.feature_selector import selected_cols

# selected_cols + base 컬럼(ID) 결합
final_cols = selected_cols + ["ID"]

# 컬럼 선택해서 복제
df_test_selected = test_df[final_cols].copy()
df_test_encoded = map_categorical_columns(df_test_selected)
for col in df_test_encoded.columns:
    if col != "ID" and df_test_encoded[col].dtype == "object":
        le = LabelEncoder()
        df_test_encoded[col] = le.fit_transform(df_test_encoded[col].astype(str))

[연회비발생카드수_B0M] 인코딩 완료
[한도증액횟수_R12M] 인코딩 완료
[이용금액대] 중간값 인코딩 완료
[할인건수_R3M] 인코딩 완료
[할인건수_B0M] 인코딩 완료
[방문횟수_PC_R6M] 인코딩 완료
[방문횟수_앱_R6M] 인코딩 완료
[방문일수_PC_R6M] 인코딩 완료


In [77]:
from modules.feature_selector import generate_e_features,generate_vif_derived_features
df_test_encoded = generate_e_features(df_test_encoded)
test_df=generate_vif_derived_features(df_test_encoded)

ImportError: cannot import name 'generate_vif_derived_features' from 'modules.feature_selector' (c:\Users\somee\Documents\GitHub\Final_project\03_modeling\99_final_model\modules\feature_selector.py)

In [65]:
import pandas as pd

# 피처 목록 CSV 로드
feature_list_df = pd.read_csv("../../data/최종_사용_피처목록.csv")

# 피처 이름 리스트 추출
selected_features = feature_list_df["selected_features"].tolist()

In [ ]:
test_df=generate_vif_derived_features(test_df)


In [50]:
# ID와 Segment가 있다면 포함
base_cols = ["ID", "Segment"]  # 또는 "target"으로 바꿔도 됨

# 실제 사용할 전체 피처
use_cols = [col for col in base_cols + selected_features if col in test_df.columns]

# 최종 피처셋 구성
df_selected = test_df[use_cols].copy()

In [51]:
X_test = df_selected.copy()

In [67]:
import xgboost as xgb
import pickle

# 모델과 피처 리스트 함께 로드
with open("./models/final_model.pkl", "rb") as f:
    loaded_model, used_features = pickle.load(f)


[08:26:16] WARNING: D:\bld\xgboost-split_1700181085428\work\src\learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [72]:
used_features

['잔액_할부_B1M',
 '여유_숙박이용금액',
 '_1순위업종_이용금액',
 '잔액_할부_B0M',
 '최종카드론_대출이율',
 '이용후경과월_할부',
 '회원여부_연체',
 '최대이용금액_일시불_R12M',
 '잔액_현금서비스_B2M',
 '이용카드수_신용',
 '잔액_카드론_B5M',
 '이용금액_할부_무이자_R12M',
 '이용금액_R3M_신용',
 'CA이자율_할인전',
 '방문일수_PC_B0M',
 '방문일수_PC_R6M',
 '평잔_일시불_3M',
 'CA한도금액',
 '이용카드수_신용체크',
 '쇼핑_온라인_이용금액',
 'RV약정청구율',
 '포인트_이용포인트_R3M',
 '이용메뉴건수_ARS_B0M',
 '이용건수_온라인_R6M',
 '정상입금원금_B5M',
 '평잔_일시불_해외_6M',
 '할부금액_무이자_3M_R12M',
 'RP건수_교통_B0M',
 '이용여부_3M_해외겸용_본인',
 '평잔_할부_3M',
 '포인트_마일리지_월적립_R3M',
 '카드이용한도금액',
 'RV일시불이자율_할인전',
 '최종카드론_대출금액',
 '교통_정비이용금액',
 '청구서발송여부_B0',
 '잔액_카드론_B3M',
 '이용카드수_신용_가족',
 '이용금액_일시불_R6M',
 '할부건수_무이자_3M_R12M',
 '변동률_RV일시불평잔',
 '당사멤버쉽_방문월수_R6M',
 '이용후경과월_할부_무이자',
 '이용금액_R3M_체크',
 '잔액_할부_해외_B0M',
 '잔액_할부_B2M',
 '인입횟수_IB_R6M',
 '변동률_할부평잔',
 '인입후경과월_ARS',
 '이용후경과월_부분무이자',
 '잔액_카드론_B0M',
 '카드론이용건수_누적',
 '이용금액_선결제_R6M',
 'RV현금서비스이자율_할인전',
 '청구서발송여부_R3M',
 '포인트_마일리지_건별_R3M',
 '이용건수_신용_R6M',
 '승인거절건수_R3M',
 '연속유실적개월수_기본_24M_카드',
 '포인트_적립포인트_R12M',
 '이용금액_일시불_R3M',
 '증감율_이용금액_일

In [71]:
# test 데이터 컬럼 맞추기
df_test_input = df_test_encoded[used_features]

# DMatrix 생성 후 예측
dtest = xgb.DMatrix(df_test_input)
pred = loaded_model.predict(dtest)

KeyError: "['신용이용건수_증가량', '일시불이용금액_최근비중', '카드론_의존도', '카드론_급증지표', '정상청구_비율', '정상청구_추세', '리볼빙_잔액비율', '방문빈도_앱', '일시불_주사용카드집중도', '이용건수_신판_R12M_minus_이용건수_신용_R12M', '이용건수_신판_R12M_div_이용건수_신용_R12M', '이용건수_일시불_R12M_minus_이용건수_신판_R12M', '이용건수_일시불_R12M_div_이용건수_신판_R12M', '이용건수_일시불_R12M_minus_이용건수_신용_R12M', '이용건수_일시불_R12M_div_이용건수_신용_R12M', '잔액_신판평균한도소진율_r3m_minus_잔액_신판최대한도소진율_r3m', '잔액_신판평균한도소진율_r3m_div_잔액_신판최대한도소진율_r3m', '잔액_카드론_B1M_minus_잔액_카드론_B0M', '잔액_카드론_B1M_div_잔액_카드론_B0M', '잔액_신판ca최대한도소진율_r3m_minus_잔액_신판ca평균한도소진율_r3m', '잔액_신판ca최대한도소진율_r3m_div_잔액_신판ca평균한도소진율_r3m', '포인트_마일리지_건별_R3M_minus_포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M_div_포인트_마일리지_건별_B0M', '방문횟수_앱_B0M_minus_방문일수_앱_B0M', '방문횟수_앱_B0M_div_방문일수_앱_B0M', '방문횟수_PC_B0M_minus_방문일수_PC_B0M', '방문횟수_PC_B0M_div_방문일수_PC_B0M', '이용금액_일시불_R3M_minus_이용금액_일시불_B0M', '이용금액_일시불_R3M_div_이용금액_일시불_B0M', '이용금액_일시불_R3M_minus_이용금액_일시불_R6M', '이용금액_일시불_R3M_div_이용금액_일시불_R6M', '이용건수_신용_R6M_minus_이용건수_신용_B0M', '이용건수_신용_R6M_div_이용건수_신용_B0M', '잔액_신판ca최대한도소진율_r6m_minus_잔